# AdaBoost

In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

In [2]:
data = pd.read_csv('./data/train.csv')

In [22]:
X = data[[
          'perc_premium_paid_by_cash_credit', 
#          'age_in_days', 
#          'Income', 
          'Count_3-6_months_late', 
          'Count_6-12_months_late', 
          'Count_more_than_12_months_late', 
          'no_of_premiums_paid', 
#          'premium',
          'application_underwriting_score',
#          'sourcing_channel'
         ]]
X['logIncome'] = np.log(data['Income'])
#X['logAge'] = np.log(data['age_in_days'])
#X['logperc_premium_paid_by_cash_credit'] = np.log(1. + data['perc_premium_paid_by_cash_credit'])
#X['standard_age'] = data['age_in_days'].apply(lambda row: np.floor(row/365.))
#X['perc'] = data['perc_premium_paid_by_cash_credit'].apply(lambda x: x if x<0.5 else x-1.)

from scipy.stats import boxcox

X['BCage_in_days'], _ = boxcox(data['age_in_days'])
#X['BCIncome'], _ = boxcox(data['Income'])

# fill nans
for name in X.columns.tolist():
    val = X[name].mean()#.iloc[0]
    X[name].fillna(val, inplace=True)
    
# predictor
y = data['renewal']

In [23]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, QuantileTransformer, Normalizer

knc = AdaBoostClassifier(n_estimators=300,
                         learning_rate=0.1
                        )

#clf = make_pipeline(StandardScaler(), knc)
clf = knc

auc_tab  = np.array([])
loss_tab = np.array([])

skf = StratifiedKFold(n_splits=10)
for train_index, test_index in skf.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    clf.fit(X_train, y_train)
    proba = clf.predict_proba(X_test)
    
    auc = roc_auc_score(y_test, proba[:,1])
    loss = log_loss(y_test, proba)
    
    auc_tab  = np.append(auc_tab, auc)
    loss_tab = np.append(loss_tab, loss)

In [24]:
print('AUC: %.8f +/- %.8f' % (np.mean(auc_tab), np.std(auc_tab)))
print('Loss: %.8f +/- %.8f' % (np.mean(loss_tab), np.std(loss_tab)))
print('-------------------------------------------')
print(knc.feature_importances_)

AUC: 0.83835340 +/- 0.00739270
Loss: 0.64150816 +/- 0.00011172
-------------------------------------------
[0.17666667 0.09       0.08333333 0.06333333 0.18666667 0.11
 0.11333333 0.17666667]
